In [1]:
### !git clone https://github.com/your-username/vascular-llm-qa.git
!git clone https://github.com/sashajh2/vascular-llm-qa.git

Cloning into 'vascular-llm-qa'...
remote: Enumerating objects: 27168, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (33/33), done.
^C


In [1]:
import pickle
import pandas as pd
import numpy as np
import pickle
import csv
import os
import time

In [2]:
from google.colab import drive, auth
from transformers import AutoModelForCausalLM, AutoTokenizer
print(auth.authenticate_user(project_id='mit-mlhc'))

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

None
Mounted at /content/drive


In [3]:
id_question = {}
with open('question_context.pickle', "rb") as f:
  id_question = pickle.load(f)

#test_ids = {'386', '385', '47.3', '44.2', '27.6', '30.4', '38.5', '14.2', '4a', '19.2', '53.5', '27', '48.3', '14.5', '54.2', '424', '43.1', '32.8', '36.4', '18.2', '15', '49.4', '59', '57.2', '371', '17.5', '50.5', '24.2', '15.7', '37.3', '418', '53.4', '48.7', '39.2', '25', '48.9', '51.2', '34.4', '36.5', '413', '23.2', '34', '401', '13', '382', '436', '38.9', '410', '447', '4b', '6b', '15.3', '47.2', '53', '40.5', '34.5', '31', '41.8', '21', '27.4', '30.5', '78', '56.ten', '50.1', '16.7', '50.ten', '37', '74', '43.8', '42.4', '31.4', '378', '414', '46.7', '28.2', '27.2', '33.4', '37.2', '444', '54.7', '6a', '22.1', '437', '12', '27.1', '24.1'}

#id_question = {k:v for k,v in id_question.items() if k in test_ids}
#



In [5]:
worksheet = gc.open('My cool spreadsheet').sheet1
rows = worksheet.get_all_values()
results = pd.DataFrame.from_records(rows)
results.columns = ["prompt_id", "question_id", "model", "answer", "response",]  # Set the first row as column names
#results.drop(results.index[0], inplace=True)  # Drop the first row since it's now the header
results

,prompt_id,question_id,model,answer,response
0,1,34.5,skumar9/Llama-medx_v3.2,B. Closure with a patch angioplasty,"A. A primary, carefully placed closure with 6-..."
1,1,386,skumar9/Llama-medx_v3.2,C. Enhancement in the peroneal artery contribu...,C Enhancement in the peroneal artery contribut...
2,1,41.4,skumar9/Llama-medx_v3.2,C. Femoropopliteal vein.,A. Long saphenous vein.
3,1,46.7,skumar9/Llama-medx_v3.2,C. Hemodynamic control of venous malformation ...,C Hemodynamic control of venous malformation a...
4,1,381,skumar9/Llama-medx_v3.2,E. Tibial artery disease,E. Tibial artery disease.
...,...,...,...,...,...
128,1,27.4,skumar9/Llama-medx_v3.2,A. Monofilament sensation testing.\n B. Achill...,A Monofilament sensation testing.\nB Monofilam...
129,1,436,skumar9/Llama-medx_v3.2,D. Pain,A. Rupture [A]
130,1,54.7,skumar9/Llama-medx_v3.2,C. Perform an infrainguinal venous thrombectom...,C Perform an infrainguinal venous thrombectomy...
131,1,4a,skumar9/Llama-medx_v3.2,D. Peripheral vascular (arterial) disease,D Peripheral vascular (arterial) disease [A]


In [4]:
p_response = {} # (prompt id, question id, model) -> model raw response
# for idx, row in results.iterrows():
#   key = row["prompt_id"], row["question_id"], row["model"]
#   p_response[key] = row["response"]
# print(len(p_response))

In [5]:
pre_prompts = {
1: "",
2: """You are a vascular surgeon responsible for teaching medical students. Provide concise, medically accurate, and ethically appropriate answers based on current standards of care. Focus only on clinical education relevant to surgery.""",
3: """You are a helpful medical knowledge assistant specializing in teaching vascular surgery to medical students. Provide useful, complete, and scientifically-grounded answers to clinical questions relevant to surgery and medical care.""",
4: """Answer the following multiple choice question from the medical domain based on these instructions. Choose one or more of the provided answer choices which are correct and output them under the heading "Answer".""",
5: """You are a helpful medical knowledge assistant specializing in teaching vascular surgery to medical students.
Answer the following multiple choice question from the medical domain based on the following instructions.
1. Select the one or more correct options and provide the option(s) under the heading "Answer".
2. Always select one or more of the provided options as the answer.
3. If the options are ambiguous or the question does not have enough context, select the ones that best answer the question.
4. If the question is not clinically relevant to vascular surgery, please respond with, "I'm sorry, I cannot answer this question." """
}

few_shot = """Question:
Which of the following is the most common site for peripheral arterial disease (PAD)?
A. Carotid artery
B. Femoral-popliteal artery
C. Renal artery
D. Subclavian artery
Answer:
B. Femoral-popliteal artery

Question:
Which of the following are common risk factors for developing an abdominal aortic aneurysm (AAA)?
A. Smoking
B. Male gender
C. Hypertension
D. Type 1 diabetes mellitus
E. Family history of AAA
Answer:
A. Smoking
B. Male gender
C. Hypertension
E. Family history of AAA
"""

pre_prompts[6] = few_shot

In [6]:
def format_mcq_example(preprompt, example):
    letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    choices = example["choices"]
    lettered_choices = [f"{letters[i]}. {choice}" for i, choice in enumerate(choices)]
    # (Instructions) - (context) - question - options
    prompt = f"Question: {example['question']}\nOptions:\n" + "\n".join(lettered_choices) + "\nAnswer:"
    context = example.get("context", [])
    if context:
        context_text = "\n".join([f"Document {i+1}: {chunk}" for i, chunk in enumerate(context)])
        prompt = f'Context:\n{context_text}{prompt}'

    explanation = example.get("explanation", "[explanation]")
    source = example.get("source", "[source]")
    if preprompt:
      prompt = f"Instructions:\n{preprompt}\n{prompt}"


    target = f"{', '.join(example['correct_answer'])} - {explanation} - {source}"
    return prompt, target





prompts = []
for prompt_id, preprompt in pre_prompts.items():
  if prompt_id != 1:
    continue
  for question in id_question.values():
    p = {"query": {"role": "user", "content": format_mcq_example(preprompt, question)[0]},
         "prompt_id": prompt_id,
         "question_id": question["question_id"]}
    prompts.append(p)

import random
random.shuffle(prompts)
print(len(prompts))

451


In [7]:
models = ["FreedomIntelligence/Apollo-0.5B", "johnsnowlabs/JSL-MedPhi2-2.7B", "skumar9/Llama-medx_v3.2", "microsoft/Phi-4-mini-instruct"]

In [8]:
import transformers
from transformers import AutoTokenizer
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import torch

In [9]:
from tqdm import tqdm
#p_response = {} # (prompt id, question id, model) -> model raw response
results = []
def run_prompts(prompts, id_question, pipe, model_name):
  for prompt in tqdm(prompts):
    key = (prompt["prompt_id"], prompt["question_id"], model_name)
    if key in p_response:
      continue # already recorded

    messages = [prompt["query"]]
    raw_response = pipe(messages, max_new_tokens=256, do_sample=True, temperature=0.1, top_k=50, top_p=0.95)
    p_response[key] = raw_response

    question = id_question[prompt["question_id"]]
    response = raw_response[0]['generated_text'][1]['content']
    answer = question['full_answers']
    entry = [*key, "RAG_100", "\n".join(answer), response, ""]
    results.append(entry)
    #worksheet.append_row(entry)
    print(key, answer, response)

In [14]:
# Load tokenizer and pipeline for
mname = models[2]
ap_tokenizer = AutoTokenizer.from_pretrained(mname)

ap_pipeline = transformers.pipeline(
    "text-generation",
    model=mname,
    tokenizer=ap_tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
run_prompts(prompts, id_question, ap_pipeline, mname)

 20%|█▉        | 88/451 [01:10<04:52,  1.24it/s]

(1, '24.4', 'skumar9/Llama-medx_v3.2') ['A. Musculotendineous sectioning is the treatment of choice in patients with a normal popliteal artery.', 'B. Vascular reconstruction should be limited to cases with stable arterial impairment.', 'C. If vascular reconstruction is planned, then the use of autologous vein is mandatory.', 'D. The posterior approach is recommended to expose all the structures causing compression.', 'E. The structure causing PAE must be sectioned completely, as incomplete sectioning may cause recurrence.'] A. Musculotendineous sectioning is the treatment of choice in patients with a normal popliteal artery.
B. Vascular reconstruction should be limited to cases with stable arterial impairment.
C. If vascular reconstruction is planned, then the use of autologous vein is mandatory.
D. The posterior approach is recommended to expose all the structures causing compression.
E. The structure causing PAE must be sectioned completely, as incomplete sectioning may cause recurre

 39%|███▉      | 176/451 [01:29<02:06,  2.18it/s]

(1, '43.6', 'skumar9/Llama-medx_v3.2') ['D. Left radial (antecubital) to basilic transposition AVF in the arm.'] E Left IJ tunneled, cuffed dual lumen hemodialysis catheter.


 40%|████      | 182/451 [04:43<09:47,  2.18s/it]

...

(1, '34.1', 'skumar9/Llama-medx_v3.2') ['D. Obtain bilateral carotid duplex scanning'] E All of the above [A] Counseling with respect to the nature of carotid territory ischaemic attacks [B] Start the patient on an antiplatelet drug, such as aspirin, and a statin [C] Counsel the patient with respect to the importance of refraining from cigarette smoking and careful control of blood pressure [D] Obtain bilateral carotid duplex scanning [E] All of the above [A] Counseling with respect to the nature of carotid territory ischaemic attacks [B] Start the patient on an antiplatelet drug, such as aspirin, and a statin [C] Counsel the patient with respect to the importance of refraining from cigarette smoking and careful control of blood pressure [D] Obtain bilateral carotid duplex scanning [E] All of the above [A] Counseling with respect to the nature of carotid territory ischaemic attacks [B] Start the patient on an antiplatelet drug, such as aspirin, and a statin [C] Counsel the patient with

100%|█████████▉| 449/451 [23:46<00:06,  3.21s/it]

(1, '5a', 'skumar9/Llama-medx_v3.2') ['D. Inpatient treatment with intravenous antibiotics and emergent surgical debridement .'] D Inpatient treatment with intravenous antibiotics and emergent surgical debridement.


100%|██████████| 451/451 [24:01<00:00,  3.20s/it]

(1, '11a', 'skumar9/Llama-medx_v3.2') ['C. Transfer to the operating room for emergent repair of the aneurysm.'] C Transfer to the operating room for emergent repair of the aneurysm.


In [21]:
print(len(results))
post_df = pd.DataFrame(results, columns=["prompt_id", "question_id", "model", "train", "answer", "response", "correct"])
post_df.to_csv('llama_post_rag_500.csv')
post_df

451


,prompt_id,question_id,model,train,answer,response,correct
0,1,39.1,skumar9/Llama-medx_v3.2,RAG_100,B. Venography\nC. Duplex scanning\nF. X-ray of...,A Pletysmography\nB Venography\nC Duplex scan...,
1,1,47.3,skumar9/Llama-medx_v3.2,RAG_100,A. Vein stripping and phlebectomy\nB. Endoveno...,A Vein stripping and phlebectomy\nB Endovenou...,
2,1,21.3,skumar9/Llama-medx_v3.2,RAG_100,B. TASC B,B TASC B\nExplanation: The TASC category of t...,
3,1,41.3,skumar9/Llama-medx_v3.2,RAG_100,E. Excision of the graft with in situ replacem...,D Excision of the graft.\nE Excision of the g...,
4,1,26.2,skumar9/Llama-medx_v3.2,RAG_100,A. Duplex scanning\nC. Computed tomography (CT...,A. Duplex scanning\nB. Arteriography\nC. Comp...,
...,...,...,...,...,...,...,...
446,1,76,skumar9/Llama-medx_v3.2,RAG_100,A. Cilostazol,A Cilostazol [A],
447,1,26.5,skumar9/Llama-medx_v3.2,RAG_100,A. Infected femoral (false) aneurysm\nE. Infec...,C Tissue scarring in the groin subsequent to r...,
448,1,34.1,skumar9/Llama-medx_v3.2,RAG_100,D. Obtain bilateral carotid duplex scanning,E All of the above [A] Counseling with respect...,
449,1,5a,skumar9/Llama-medx_v3.2,RAG_100,D. Inpatient treatment with intravenous antibi...,D Inpatient treatment with intravenous antibio...,


In [16]:
# Load tokenizer and pipeline for phi
ap_tokenizer = AutoTokenizer.from_pretrained(models[2])

ap_pipeline = transformers.pipeline(
    "text-generation",
    model=models[2],
    tokenizer=ap_tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
run_prompts(prompts, id_question, ap_pipeline, models[2])

  3%|▎         | 13/451 [00:17<09:42,  1.33s/it]

(1, '15.1', 'skumar9/Llama-medx_v3.2') ['A. Secure an airway, commence oxygen therapy and obtain adequate intravenous (IV) access.', 'D. Transfer the patient to theatre for definitive management during primary resuscitation.'] A. Secure an airway, commence oxygen therapy and obtain adequate intravenous (IV) access.


  3%|▎         | 15/451 [00:23<12:17,  1.69s/it]

(1, '368', 'skumar9/Llama-medx_v3.2') ['B. Hypoglossal Nerve.'] B Hypoglossal Nerve.


  6%|▋         | 29/451 [00:43<10:24,  1.48s/it]

...

(1, '49.7', 'skumar9/Llama-medx_v3.2') ['B. It is administered around the vein and helps prevent conduction of heat to surrounding tissue.', 'C. It increases the efficacy of treatment by compressing the vein and causing vasospasm.'] B It is administered around the vein and helps prevent conduction of heat to surrounding tissue.


 76%|███████▌  | 341/451 [1:13:27<21:07, 11.52s/it]

(1, '16.4', 'skumar9/Llama-medx_v3.2') ['B. Computerized tomography of head, neck, chest, abdomen and pelvis'] B Computerized tomography of head, neck, chest, abdomen and pelvis
Discussion: The patient has a complex pattern of injury including a tension pneumothorax, open mid-shaft fracture of the right femur and abdominal
injury. The most appropriate next investigation is a computerized tomography (CT) scan of the head, neck, chest, abdomen and pelvis to assess the
extent of the injuries and to guide further management. This will provide a detailed assessment of the thoracic, abdominal and pelvic organs, as well
as the musculoskeletal system. The CT scan will also help to identify any other injuries that may not be immediately apparent. The patient's
hemodynamics have stabilized with fluid resuscitation and blood transfusion, but further intervention is required to manage the tension pneumothorax
and the abdominal injury. The CT scan will provide the necessary information to guide the

 78%|███████▊  | 353/451 [1:14:38<15:11,  9.30s/it]

(1, '47.1', 'skumar9/Llama-medx_v3.2') ['A. High-flow arteriovenous shunting'] A High-flow arteriovenous shunting
B High-flow arteriovenous shunting
C High-flow arteriovenous shunting
D High-flow arteriovenous shunting
Correct Answer: assistant
A High-flow arteriovenous shunting
B High-flow arteriovenous shunting
C High-flow arteriovenous shunting
D High-flow arteriovenous shunting
Correct Answer: 


In [ ]:
!pip install google-cloud-sdk

ERROR: Could not find a version that satisfies the requirement google-cloud-sdk (from versions: none)
ERROR: No matching distribution found for google-cloud-sdk


In [ ]:
!gcloud config set project macro-gadget-458902-i2

Updated property [core/project].


In [ ]:
!gcloud config list project

[core]
project = macro-gadget-458902-i2

Your active configuration is: [default]


In [ ]:
from google.cloud import storage

# Create a client object
client = storage.Client()

In [ ]:
!gcloud beta billing projects describe macro-gadget-458902-i2

billingAccountName: billingAccounts/010968-A1D9E2-69C8E5
billingEnabled: true
name: projects/macro-gadget-458902-i2/billingInfo
projectId: macro-gadget-458902-i2


In [ ]:
#!gcloud auth login
!gcloud config set project macro-gadget-458902-i2

Updated property [core/project].


In [ ]:
bucket = client.bucket('mlhcbucket')

In [ ]:
bucket.exists()

True

In [ ]:
blobs = bucket.list_blobs()
for blob in blobs:
  print(blob.name)

In [ ]:
print(blobs)

In [ ]:
blob = bucket.blob('gemini_batch_requests.jsonl')

In [ ]:
blob

<Blob: mlhcbucket, gemini_batch_requests.jsonl, None>

In [ ]:
# run RAG examples on gemini

# Set up Google Vertex API call
# NOTE -- can comment this out
# (not needed for hugging face models)
import vertexai
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Image,
    Part,
    SafetySetting,
)
from vertexai.batch_prediction import BatchPredictionJob
import json
from google.cloud import storage

# replace with project ID from Google Cloud Platform
PROJECT_ID = "macro-gadget-458902-i2"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
def df_to_jsonl_gcs(prompts, bucket_name, blob_name):
    """Converts a DataFrame to JSONL and uploads to Google Cloud Storage.

    Args:
        df: Pandas DataFrame with a 'text' column.
        bucket_name: Name of your Google Cloud Storage bucket.
        blob_name: Desired name for the JSONL file on GCS.
    """



    # Write JSONL data to a string buffer
    jsonl_data = ""
    for prompt in prompts:
        text = prompt["query"]["content"]
        json_data = {
            "prompt_id": prompt["prompt_id"],
            "model": "gemini-2.0-flash-001",
            "question_id": prompt["question_id"],
            "request": {
                "contents": [
                    {
                        "role": "user",
                        "parts": [{"text": text}]
                    }
                ],
                "generationConfig": {"temperature": 0.1, "maxOutputTokens": 256},

            }
        }
        jsonl_data += json.dumps(json_data) + '\n'
    return jsonl_data

    # Upload the JSONL data to GCS
    blob.upload_from_string(jsonl_data, content_type='application/jsonl')
    print(f"JSONL file uploaded to gs://{bucket_name}/{blob_name}")

    return f"gs://{bucket_name}/{blob_name}"


    # Specify a custom bucket name
BUCKET_NAME = 'mlhcbucket'

data = df_to_jsonl_gcs(prompts, BUCKET_NAME, 'gemini_batch_requests.jsonl')


In [ ]:
blob.upload_from_string(data, content_type='application/jsonl')


Forbidden: 403 POST https://storage.googleapis.com/upload/storage/v1/b/mlhcbucket/o?uploadType=multipart: {
  "error": {
    "code": 403,
    "message": "The billing account for the owning project is disabled in state closed",
    "errors": [
      {
        "message": "The billing account for the owning project is disabled in state closed",
        "domain": "global",
        "reason": "accountDisabled",
        "locationType": "header",
        "location": "Authorization"
      }
    ]
  }
}
: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>)

In [ ]:
import json

def df_to_jsonl_file(prompts, filename):
    """Converts a list of prompts to JSONL and writes to a local file.

    Args:
        prompts: A list of prompts.
        filename: The name of the file to write the JSONL data to.
    """

    with open(filename, 'w') as f:
        for prompt in prompts:
            text = prompt["query"]["content"]
            json_data = {
                "prompt_id": prompt["prompt_id"],
                "model": "gemini-2.0-flash-001",
                "question_id": prompt["question_id"],
                "request": {
                    "contents": [
                        {
                            "role": "user",
                            "parts": [{"text": text}]
                        }
                    ],
                    "generationConfig": {"temperature": 0.1, "maxOutputTokens": 256},

                }
            }
            f.write(json.dumps(json_data) + '\n')

    print(f"JSONL data written to {filename}")


# Example usage
filename = 'gemini_batch_requests.jsonl'  # Choose your desired filename
df_to_jsonl_file(prompts, filename)  # Assuming 'prompts' is your list of prompts

JSONL data written to gemini_batch_requests.jsonl


In [ ]:
input_uri = 'gs://mlhcbucket/gemini_batch_requests.jsonl'
output_uri = f"gs://{BUCKET_NAME}/batch-prediction/"

# Submit a batch prediction job with Gemini model
batch_prediction_job = BatchPredictionJob.submit(
    source_model="gemini-2.0-flash-001",
    #source_model="gemini-1.5-flash-001",
    #source_model="jsl-medphi2-2.7b",
    #"projects/552077056809/locations/us-central1/endpoints/1130919177423421440",
    input_dataset=input_uri,
    output_uri_prefix=output_uri,
)

# Check job status
print(f"Job resource name: {batch_prediction_job.resource_name}")
print(f"Model resource name with the job: {batch_prediction_job.model_name}")
print(f"Job state: {batch_prediction_job.state.name}")

# Refresh the job until complete
while not batch_prediction_job.has_ended:
    time.sleep(5)
    batch_prediction_job.refresh()

# Check if the job succeeds
if batch_prediction_job.has_succeeded:
    print("Job succeeded!")
else:
    print(f"Job failed: {batch_prediction_job.error}")

# Check the location of the output
print(f"Job output location: {batch_prediction_job.output_location}")

# # Example response:
# #  Job output location: gs://your-bucket/gen-ai-batch-prediction/prediction-model-year-month-day-hour:minute:second.12345


# # NOTE:
# # After job finishes running, copied data over to my drive and loaded it
# # into a pandas dataframe
# # Job took 2 min 55 sec

INFO:vertexai.batch_prediction._batch_prediction:BatchPredictionJob created. Resource name: projects/201055451681/locations/us-central1/batchPredictionJobs/4229961024649822208
INFO:vertexai.batch_prediction._batch_prediction:To use this BatchPredictionJob in another session:
INFO:vertexai.batch_prediction._batch_prediction:job = batch_prediction.BatchPredictionJob('projects/201055451681/locations/us-central1/batchPredictionJobs/4229961024649822208')
INFO:vertexai.batch_prediction._batch_prediction:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/4229961024649822208?project=201055451681


Job resource name: projects/201055451681/locations/us-central1/batchPredictionJobs/4229961024649822208
Model resource name with the job: publishers/google/models/gemini-2.0-flash-001
Job state: JOB_STATE_PENDING
Job succeeded!
Job output location: gs://mlhcbucket/batch-prediction/prediction-model-2025-05-05T03:28:55.083682Z


In [ ]:
# NOT NEEDED FOR HUGGING FACE MODELS
# load predictions from vertex
#predictions_fname = '/content/drive/MyDrive/llm_eval_vertex_response.jsonl'
predictions_fname = 'gemini_post_rag.jsonl'
predictions_df = pd.read_json(path_or_buf=predictions_fname, lines=True)


In [ ]:
# NOT NEEDED FOR HUGGING FACE
def extract_text(response):
  """Extracts text from the response dictionary, handling potential KeyError."""
  try:
    return response['candidates'][0]['content']['parts'][0]['text']
  except (KeyError, IndexError, TypeError):
    # Handle cases where 'parts' key is missing or empty
    print(response)
   # Or any other appropriate default value
    return ''

predictions_df['summary'] = predictions_df['response'].apply(extract_text)
predictions_df

,question_id,model,prompt_id,status,processed_time,request,response,summary
0,419,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.720000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.16134893379...,The correct answer is **A. Stanford A dissecti...
1,32.1,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.727000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.17606310376...,"Okay, let's analyze the provided documents and..."
2,41.6,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.728000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.17053280998...,"Based on the provided context, here's a breakd..."
3,401,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.728000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.07246243127...,"The answer is **A. Full leaded face shield, le..."
4,52.5,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.733000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.16047630310...,Based on the provided documents and the patien...
...,...,...,...,...,...,...,...,...
446,40.4,gemini-2.0-flash-001,1,,2025-05-05 03:32:25.581000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.13143135519...,"Based on the provided documents, here's an ana..."
447,57.2,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.722000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.12899301147...,"Based on the provided documents, here's an ana..."
448,57.3,gemini-2.0-flash-001,1,,2025-05-05 03:32:12.325000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.15896653559...,"Based on the provided documents, here's an ana..."
449,373,gemini-2.0-flash-001,1,,2025-05-05 03:32:15.516000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.18681633472...,"Based on the provided documents, the correct a..."


In [ ]:
appended = []
# NOT NEEDED FOR HUGGING FACE
# write response back to spreadsheet
# vertex_df.drop(columns='id', inplace=True)
# vertex_df

#worksheet = gc.open('test_sheet').sheet1
#cols = predictions_df.columns.tolist()
#worksheet.append_row(["prompt_id", "question_id", "model", "answer", "response", "correct"])
print(len(appended))
for index, row in predictions_df.iterrows():
  question = id_question[row["question_id"]]
  answer = "\n".join(question['full_answers'])
  r = [row["prompt_id"], row["question_id"], row["model"], answer, row["summary"], ""]
  if r in appended:
    continue
  print(r)
  #worksheet.append_row(r)
  appended.append(r)
# dumped it to a different sheet, then copy and pasted it manually to avoid overwriting the actual spreadsheet
# manually evaluated responses


0
[1, '419', 'gemini-2.0-flash-001', 'A. Stanford A dissection.\nC. de Bakey I dissection.', "The correct answer is **A. Stanford A dissection.**\n\nHere's why:\n\n*   **Stanford Classification:** The Stanford classification system divides aortic dissections into two types:\n    *   **Type A:** Involves the ascending aorta (regardless of the site of origin or extent).\n    *   **Type B:** Does not involve the ascending aorta.\n*   **de Bakey Classification:** The de Bakey classification system divides aortic dissections into three types:\n    *   **Type I:** Originates in the ascending aorta and extends at least to the aortic arch, possibly further down the descending aorta.\n    *   **Type II:** Originates in and is confined to the ascending aorta.\n    *   **Type III:** Originates in the descending aorta and extends distally (Type IIIa) or both proximally and distally (Type IIIb).\n\nIn this case, the dissection involves the ascending aorta, aortic arch, and descending aorta. Since t

In [ ]:
resdf = pd.DataFrame(appended, columns=["prompt_id", "question_id", "model", "answer", "response", "correct"])
resdf
resdf.to_csv('vertex_post_eval.csv')

In [10]:
parsed_rows = []

raw1 = ''
# with open('llama_100_out.txt', 'r') as f:
#   raw1 = f.read()

raw5 = ''
with open('llama5002.txt', 'r') as f:
  raw5 = f.read()


In [11]:
lines = raw5.split("\n\n(1, '")[1:]

print(len(lines))
line = lines[0]


for line in lines:
  qid = line[:line.index(",")].strip().replace("'", '')
  model = line[line.index("', '")+3:line.index("')")].replace("'", '')
  response = line[line.index(']')+1:]

  assert qid in id_question
  assert model in models
  key = (1, qid, model)
  p_response[key] = response

len(p_response)

429


429

In [12]:
results = []
model_name = model
for prompt in prompts:
    key = (prompt["prompt_id"], prompt["question_id"], model_name)
    if key not in p_response:
      continue
    question = id_question[prompt["question_id"]]
    response = p_response[key]
    answer = question['full_answers']
    entry = [*key, "RAG_100", "\n".join(answer), response, ""]
    results.append(entry)

In [13]:
len(results)

429

In [46]:
post_df = pd.DataFrame(results, columns=["prompt_id", "question_id", "model", "train", "answer", "response", "correct"])
post_df.to_csv('llama_post_rag_100.csv')